In [13]:
import pandas as pd
import openpyxl
from openai import OpenAI, OpenAIError
import re
import time
import openpyxl

#Cargamos el archivo de Excel en un DataFrame
problemas = pd.read_excel('/content/drive/MyDrive/Traducción_DataSets.xlsx')
dataset = pd.DataFrame(problemas)

#Inicializamos el cliente de OpenAI (corregido)
client = OpenAI(api_key='')

#Prueba de conexión con OpenAI
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "¿Cuánto es 2+2?"}]
)
print("Prueba API:", response.choices[0].message.content)

#Función para resolver los problemas aritméticos
def solver(cuerpo, pregunta, prompt):
    try:
        if pd.isna(cuerpo) or pd.isna(pregunta):
            return "El cuerpo o la pregunta está vacío."

        respuesta = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Eres un asistente que resuelve problemas matemáticos."},
                {"role": "user", "content": f"{cuerpo} {pregunta} {prompt}"}
            ]
        )

        #Acceso al contenido de la respuesta
        return respuesta.choices[0].message.content

    except OpenAIError as e:
        return f"Error en la API de OpenAI: {str(e)}"
    except Exception as e:
        return f"Error desconocido: {str(e)}"

#Tomamos un subconjunto del Dataset
dataset_n = dataset.iloc[:,:]

answers = pd.DataFrame(dataset_n)

#Aplicamos la función solver a cada fila
answers['Solucion'] = answers.apply(lambda row: solver(row.get('Cuerpo', ''), row.get('Pregunta', ''), row.get('Prompt')), axis=1)

def numero(respuesta):

  sol = re.findall(r'-?\d+(?:\.\d+)?', respuesta)
  if sol:
    return sol[-1]
  else:
    return None

answers['sol_num'] = answers['Solucion'].apply(numero)

answers['Respuesta_GPT'] = answers['sol_num'].round(2)

#Configuramos para mostrar todo el contenido de las columnas
pd.set_option('display.max_colwidth', None)

answers['Acierto'] = answers['Respuesta_GPT'].astype(float) == answers['Respuesta'].astype(float)

pd.set_option('display.max_rows', None)

evaluacion = pd.DataFrame(answers[['Respuesta_GPT','Respuesta','Acierto']])

#print(evaluacion)

correct = answers['Acierto'].sum()

print(f'Número de respuestas correctas: {correct}')

c = len(answers)

accuracy = correct / c*100

print(f'Porcentaje de aciertos: {accuracy:.2f}%')

evaluacion.to_excel('/content/drive/MyDrive/evaluacion.xlsx', index=False)


Prueba API: 2+2 es igual a 4.
Número de respuestas correctas: 66
Porcentaje de aciertos: 66.00%
